<a href="https://colab.research.google.com/github/gojiteji/Hatena-Textbook/blob/master/%E3%81%84%E3%81%84%E6%84%9F%E3%81%98txtt21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 777 /root/.kaggle/kaggle.json
!kaggle competitions download -c nlp-getting-started

  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 62.1MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.9MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 22.3MB/s]


In [1]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import numpy as np
import xgboost as xgb
import torch
import torch.nn as nn
import re
from nltk.tokenize import sent_tokenize, word_tokenize ,TweetTokenizer
import torch.optim as optimizers
from torch.utils.data import DataLoader,TensorDataset
!pip install tqdm
from tqdm import tqdm
warnings.simplefilter('ignore')
import nltk
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
#辞書構造
class Vocabulary():
    def __init__(self,disabletokenize=False):
        self.w2i={}
        self.i2w={}
        self.oov_char='<unk>'
        self.pad='<pad>'
        self.special_chars = [self.pad,self.oov_char]
        self.data = ""
        self._words=set()
        self.tokenize=disabletokenize
        self.tknzr = TweetTokenizer()
    def update(self,text):
        if(self.tokenize):
            self._words.update(text)
        else:
            self.data=self.tknzr.tokenize(text)
            #self.data=word_tokenize(text)
            self._words.update(self.data)
        self.w2i = {w: (i + len(self.special_chars)) for i, w in enumerate(self._words)}

        self.i2w = {i: w for w, i in self.w2i.items()}
        self.w2i['<pad>'] = 0
        self.i2w[0] = '<pad>'
        self.w2i['<unk>'] = 1
        self.i2w[1] = '<unk>'


    def encode(self,words):
        output=[]
        if(self.tokenize):
            pass
        else:
            #words=word_tokenize(words)
            words=self.tknzr.tokenize(words)
        for word in words:
            #辞書になし
            if word not in self.w2i:
                index = self.w2i[self.oov_char]#既存の<unk>を返す
            else:
            #辞書にあり
                index = self.w2i[word]#idを引っ張ってくる
            output.append(index)
        return output


    def decode(self,indexes):#使いどころないけど確認用
        out=[]
        for index in indexes:
            out.append(self.i2w[index])
        return out



df = pd.read_csv('train.csv')
keyword_voc=Vocabulary(disabletokenize=True)
keywords = np.array(df.iloc[:,[1]].fillna('nan'))
keywords=keywords.reshape(len(keywords))
keyword_voc.update(list(keywords))#キーワードを辞書に登録
text_voc=Vocabulary()
text = np.array(df.iloc[:,[3]])
text=text.reshape(len(text))

x=np.stack([text, keywords], 1)
y = np.array(df.iloc[:,4])
raw_X_train, X_test, raw_y_train, y_test = train_test_split(x, y,test_size=0.3,shuffle=True,stratify=y)

#trainデータを辞書に登録
for t in raw_X_train:
    text_voc.update(t[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
#testデータをtrainデータ内の単語でid化
maxlen=158
id_=0
x_test=np.array([[0]*maxlen]*len(X_test))
x_test_key=np.array([0]*len(X_test))
for t in X_test:
    tmp=text_voc.encode(t[0])
    tmp2=keyword_voc.encode([t[1]])
    for i in range(maxlen-len(tmp)):
        tmp.append(0)
    x_test[id_]=tmp
    x_test_key[id_]=tmp2[0]
    id_=id_+1

In [0]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self,d_k,device='cuda'):
        super().__init__()
        self.device = device
        self.scaler = np.sqrt(d_k)
    def forward(self, q, k, v, mask=None):
        '''
        # Argument
            q, k, v: (batch, sequence, out_features)
            mask:    (batch, sequence)
        '''
        score = torch.einsum('ijk,ilk->ijl', (q, k)) / self.scaler
        score = score - torch.max(score, dim=-1, keepdim=True)[0]
        score = torch.exp(score)
        if mask is not None:
            if len(mask.size()) == 2:
                mask = mask.unsqueeze(1).repeat(1, score.size(1), 1)
            score.data.masked_fill_(mask, 0)
        a = score / torch.sum(score, dim=-1, keepdim=True)
        c = torch.einsum('ijk,ikl->ijl', (a, v))
        return c


class PositionalEncoding(nn.Module):
    def __init__(self, output_dim,maxlen=10000,device='cuda'):
        super().__init__()
        self.device = device
        self.output_dim = output_dim
        self.maxlen = maxlen
        pe = self.initializer()
        self.register_buffer('pe', pe)
    def forward(self, x, mask=None):
        pe = self.pe[:x.size(1), :].unsqueeze(0)
        return x + pe
    def initializer(self):
        pe = \
            np.array([[pos / np.power(100, 2 * (i // 2) / self.output_dim)
                       for i in range(self.output_dim)]
                      for pos in range(self.maxlen)])
        pe[:, 0::2] = np.sin(pe[:, 0::2])
        pe[:, 1::2] = np.cos(pe[:, 1::2])
        return torch.from_numpy(pe).float()


class MultiHeadAttention(nn.Module):
    def __init__(self,h,d_model,device='cuda'):
        super().__init__()
        self.h = h
        self.d_model = d_model
        self.d_k = d_k = d_model // h
        self.d_v = d_v = d_model // h
        self.device = device
        self.W_q = nn.Parameter(torch.Tensor(h,d_model, d_k))
        self.W_k = nn.Parameter(torch.Tensor(h,d_model,d_k))
        self.W_v = nn.Parameter(torch.Tensor(h,d_model,d_v))

        nn.init.xavier_normal_(self.W_q)
        nn.init.xavier_normal_(self.W_k)
        nn.init.xavier_normal_(self.W_v)

        self.attn = ScaledDotProductAttention(d_k)
        self.linear = nn.Linear((h * d_v), d_model)
        nn.init.xavier_normal_(self.linear.weight)

    def forward(self, q, k, v, mask=None):
        '''
        # Argument
            q, k, v: (batch, sequence, out_features)
            mask:    (batch, sequence)
        '''
        batch_size = q.size(0)
        q = torch.einsum('hijk,hkl->hijl',(q.unsqueeze(0).repeat(self.h, 1, 1, 1),self.W_q))
        k = torch.einsum('hijk,hkl->hijl',(k.unsqueeze(0).repeat(self.h, 1, 1, 1),self.W_k))
        v = torch.einsum('hijk,hkl->hijl',(v.unsqueeze(0).repeat(self.h, 1, 1, 1),self.W_v))
        q = q.view(-1, q.size(-2), q.size(-1))
        k = k.view(-1, k.size(-2), k.size(-1))
        v = v.view(-1, v.size(-2), v.size(-1))

        if mask is not None:
            multiples = [self.h] + [1] * (len(mask.size()) - 1)
            mask = mask.repeat(multiples)

        c = self.attn(q, k, v, mask=mask)
        c = torch.split(c, batch_size, dim=0)
        c = torch.cat(c, dim=-1)

        out = self.linear(c)

        return out

class FFN(nn.Module):
    def __init__(self, d_model, d_ff,device='cuda'):
        super().__init__()
        self.l1 = nn.Linear(d_model, d_ff)
        self.a1 = nn.ReLU()
        self.l2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        h = self.l1(x)
        h = self.a1(h)
        y = self.l2(h)
        return y

class Attentions(nn.Module):
    def __init__(self,depth_source,N=6,h=8,d_model=512,d_ff=2048,p_dropout=0.1,maxlen=128,device='cuda'):
        super().__init__()
        self.device = device
        self.embedding = nn.Embedding(depth_source,d_model, padding_idx=0)
        self.pe = PositionalEncoding(d_model, maxlen=maxlen)
        self.encoder_layers = nn.ModuleList([AttentioLayer(h=h,d_model=d_model,d_ff=d_ff,p_dropout=p_dropout,maxlen=maxlen,device=device) for _ in range(N)])
    def forward(self, x, mask=None):
        x = self.embedding(x)
        y = self.pe(x)
        for encoder_layer in self.encoder_layers:
            y = encoder_layer(y, mask=mask)
        return y
class AttentioLayer(nn.Module):
    def __init__(self,h=8,d_model=512,d_ff=2048,p_dropout=0.1,maxlen=128,device='cuda'):
        super().__init__()
        self.attn = MultiHeadAttention(h, d_model)
        self.dropout1 = nn.Dropout(p_dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = FFN(d_model, d_ff)
        self.dropout2 = nn.Dropout(p_dropout)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        h = self.attn(x, x, x, mask=mask)
        h = self.dropout1(h)
        h = self.norm1(x + h)
        y = self.ff(h)
        y = self.dropout2(y)
        y = self.norm2(h + y)
        return y



class AttentionNN(nn.Module):
    def __init__(self,depth_source,depth_key,N=6,h=8,d_model=512,d_key=224,d_ff=2048,d_off=200,p_dropout=0.3,maxlen=128,device='cuda'):
        """
        (形態素数＝ベクトル数)
        depth_source:全ツイートの形態素数
        N=6:エンコーダ/デコーダのレイヤを重ねる数
        h=8:Multi head attentionにける、attentionの数
        d_model=512:埋め込み次元
        d_model=512:キーの埋め込み次元
        d_ff=2048:フィードフォワード(通常のNNの順伝播)
        p_dropout=0.3:ドロップアウト率
        maxlen=128:1文の含む最大形態素数
        """
        super().__init__()
        self.device = device
        self.embedding_key = nn.Embedding(depth_key,d_key, padding_idx=0)
        self.d_model=d_model
        self.maxlen = maxlen
        self.Attantions = Attentions(depth_source=33550,N=N,h=h,d_model=d_model,d_ff=d_ff,p_dropout=p_dropout,maxlen=maxlen,device=device)
        
        #Attn model
        #self.l1_= nn.Linear(d_model*maxlen, d_off)
        #self.a1_= nn.ReLU()
        #self.d1_= nn.Dropout(p=p_dropout)

        #key model
        self.l1__= nn.Linear(d_key, d_off)
        self.a1__= nn.ReLU()
        self.d1__= nn.Dropout(p=p_dropout)
        self.l2__= nn.Linear(d_off, d_off)
        self.a2__= nn.ReLU()
        self.d2__= nn.Dropout(p=p_dropout)

        #concated model
        self.l1= nn.Linear(d_key+d_model*maxlen, d_off)
        self.a1= nn.ReLU()
        self.d1= nn.Dropout(p=p_dropout)
        self.l2= nn.Linear(d_off, d_off)
        self.a2= nn.ReLU()
        self.d2= nn.Dropout(p=p_dropout)
        self.l3= nn.Linear(d_off, d_off)
        self.a3= nn.ReLU()
        self.d3= nn.Dropout(p=p_dropout)


        self.linear_out = nn.Linear(d_off, 1)
        self.activation_out = nn.Sigmoid()

        nn.init.xavier_normal_(self.linear_out.weight)
        #nn.init.kaiming_normal_(self.l1_.weight)
        nn.init.kaiming_normal_(self.l1.weight)
        nn.init.kaiming_normal_(self.l2.weight)
        #nn.init.kaiming_normal_(self.l1__.weight)
        #nn.init.kaiming_normal_(self.l2__.weight)

        nn.init.kaiming_normal_(self.l3.weight)
        #nn.init.kaiming_normal_(self.l4.weight)

    def sequence_mask(self, x):
        return x.eq(0)
    def forward(self,source, key):
        key=self.embedding_key(key)
        mask_source = self.sequence_mask(source)
        hs = self.Attantions(source, mask=mask_source)
        a_out=hs.reshape(len(source),self.d_model*maxlen)
        #a_out=self.l1_(hs.reshape(len(source),self.d_model*maxlen))
        #a_out=self.a1_(a_out)
        #a_out=self.d1_(a_out)

        b_out =key
        #b_out =self.l1__(key)
        #b_out =self.a1__(b_out)        
        #b_out =self.d1__(b_out)
        #b_out =self.l2__(b_out)
        #b_out =self.a2__(b_out)
        #b_out =self.d2__(b_out)       
        out = self.l1(torch.cat([a_out,b_out],dim=1))
        out = self.a1(out)
        out = self.d1(out)
        out = self.l2(out)
        out = self.a2(out)
        out = self.d2(out)
        out = self.l3(out)
        #out = self.a3(out)
        #out = self.d3(out)
        #out = self.l4(out)
        #out = self.a4(out)
        #out = self.d4(out)

        #
        out = self.linear_out(out)
        out = self.activation_out(out)
        return out


In [0]:
model=AttentionNN(depth_source=27330,depth_key=224,N=5,h=4,d_model=400,d_key=224,d_ff=500,d_off=400,p_dropout=0.2,maxlen=maxlen,device='cuda').to('cuda')
optimizer = optimizers.Adam(model.parameters(),lr=0.000001,amsgrad=True)
criterion = nn.BCELoss()
new=True

In [5]:
if(new):
    training_loss = []
    trainaccs=[]
    accs=[]
    new=False
#モデルの学習
epochs=3000

#テストデータをダウンサンプリング
x_test_=np.concatenate([x_test.reshape(len(x_test),maxlen), x_test_key.reshape(len(x_test),1)],1)
data=torch.LongTensor(x_test_.astype(np.float32)).to('cuda')
a=0
for epoch in range(epochs):
    """
    【データローダの設定】
    trainデータはダウンサンプリングされるので、そのまま使うと14%ほど、学習データが失われる。
    したがって、、エポックごとにraw_trainからサンプリングしたものをデータローダに設定
    """
    X_train, y_train = rus.fit_sample(raw_X_train.reshape(-1, 2) ,raw_y_train.reshape(-1, 1) )
    maxlen=158
    id_=0
    x_train=np.array([[0]*maxlen]*len(X_train))
    x_train_key=np.array([0]*len(X_train))
    for t in X_train:
        tmp=text_voc.encode(t[0])
        tmp2=keyword_voc.encode([t[1]])
        for i in range(maxlen-len(tmp)):
            tmp.append(0)
        x_train[id_]=tmp
        x_train_key[id_]=tmp2[0]
        id_=id_+1
    """データ水増し"""
    unks=0#unkの量
    max_unks=(x_train.size - sum(sum(x_train==0)))*0.25#paddingの量
    max_row=x_train.shape[0]
    max_colum=x_train.shape[1]
    while(unks<max_unks):
        row=int(np.random.rand()*max_row)
        colum=int(np.random.rand()*max_colum)
        if(x_train[row,colum] is not 0):#paddingじゃなければunkに置き換え
            x_train[row,colum]=1
            unks=unks+1

    batch_size=len(x_train)// 25
    x_in=np.concatenate([x_train.reshape(len(x_train),maxlen), x_train_key.reshape(len(x_train),1)], 1)
    x=torch.LongTensor(x_in.astype(np.float32)).to('cuda')
    y=torch.tensor(y_train.reshape(len(y_train),1).astype(np.float32)).to('cuda')
    dataset = TensorDataset(x,y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    #send_line_notification(i)
    runnning_loss = 0.0
    predis1=np.array([])
    predis2=np.array([])
    teachers=np.array([])
    model.train()
    print("begin training")
    i=0
    for x,y in dataloader:
        i=i+1
        model.zero_grad()
        predi=model(x[:,:-1],x[:,-1])
        loss = criterion(predi,y)
        loss.backward()
        optimizer.step()
    
        runnning_loss += loss.item()
        predis1=np.append(predis1,predi.detach().cpu().numpy())
        teachers=np.append(teachers,y.cpu())
    max_acc=0
    acc=0
    max_thread=50
    bestans=[0]
    if(True):#for thread in range(40,99,3):
        thread=50
        predis2=np.where(predis1 > thread/100, 1, 0)
        acc = accuracy_score(teachers,predis2)
        if(True):#acc>max_acc):
            max_acc=acc
            max_thread=thread/100
            bestans =predis2
    if(epoch%1==0):
        print("epoch:",epoch)
        print("loss:",runnning_loss)
        print("*train data acc:",max_acc," thread:",max_thread)
        a=sum(bestans)
    trainaccs.append(max_acc)
    training_loss.append(runnning_loss)
    predis1=np.array([])
    predis2=np.array([])
    teachers=np.array([])
    true_indexes=[]
    false_indexes=[]
    model.eval()
    us_len=sum(y_test)#この大きさに合わせてunder sampling
    for j in range(data.shape[0]):
        y_pred=model(data[j:j+1,:-1],data[j:j+1,-1])
        predis1=np.append(predis1,y_pred.detach().cpu().numpy())
        teachers=np.append(teachers,y_test[j])
        #アンダーサンプリング用
        if((len(true_indexes)<=us_len) and (y_test[j])):
            true_indexes.append(j)
        elif((len(false_indexes)<=us_len) and (not y_test[j])):
            false_indexes.append(j)
    c=0
    acc=0
    acc_us=0
    max_thread=0
    bestans=[0]
    #for thread in range(40,99,3):
    predis2=np.where(predis1 > 50/100, 1, 0)
    acc = accuracy_score(teachers,predis2)
    acc_us1 = accuracy_score(teachers[true_indexes],predis2[true_indexes])
    acc_us2 = accuracy_score(teachers[false_indexes],predis2[false_indexes])
    #    if(acc>max_acc):
    max_acc=acc
    max_thread=50/100
    bestans=predis2
    if(epoch%1==0):
        print("*test_us data acc:",(acc_us1 + acc_us2)/2," thread:",max_thread," TP:",acc_us1," FN:",acc_us2)
        print("mean train:",a,"/",len(x_train),"=",a/len(x_train))
        print("mean test_us:",sum(predis2[true_indexes])+sum(predis2[false_indexes]),"/",len(true_indexes)+len(false_indexes),"=",(sum(predis2[true_indexes])+sum(predis2[false_indexes]))/(len(true_indexes)+len(false_indexes)))
        print()
        print("*test data acc:",max_acc," thread:",max_thread)
        print("mean test:",sum(bestans),"/",len(x_test),"=",sum(bestans)/len(x_test))
        print("--------------------")
    accs.append(max_acc)
    if(max_acc>0.81):#testデータに対し目標値を超えたら切る
        break

begin training
epoch: 0
loss: 22.542653381824493
*train data acc: 0.4982532751091703  thread: 0.5
*test_us data acc: 0.5287566617047736  thread: 0.5  TP: 0.4780835881753313  FN: 0.5794297352342159
mean train: 2236 / 4580 = 0.4882096069868996
mean test_us: 882 / 1963 = 0.4493122771268467

*test data acc: 0.5328371278458844  thread: 0.5
mean test: 1024 / 2284 = 0.44833625218914186
--------------------
begin training
epoch: 1
loss: 20.2410985827446
*train data acc: 0.5172489082969433  thread: 0.5
*test_us data acc: 0.5552830666575318  thread: 0.5  TP: 0.5759429153924567  FN: 0.5346232179226069
mean train: 2285 / 4580 = 0.49890829694323147
mean test_us: 1022 / 1963 = 0.5206316861946001

*test data acc: 0.5490367775831874  thread: 0.5
mean test: 1179 / 2284 = 0.5161996497373029
--------------------
begin training
epoch: 2
loss: 20.091830730438232
*train data acc: 0.5024017467248908  thread: 0.5
*test_us data acc: 0.5665194707590866  thread: 0.5  TP: 0.6442405708460754  FN: 0.488798370672097

KeyboardInterrupt: ignored

In [0]:
plt.plot(training_loss)
plt.ylim(0.,max(training_loss))
plt.show()
plt.ylim(0.,1.)
plt.grid(which='major',color='gray',linestyle='-')
plt.plot(trainaccs,color="blue",label="train acc")
plt.plot(accs,color="orange",label="test acc")
plt.legend()
plt.show()

In [0]:
max_threshold=0
max_tacc=0
accplot=[]
accplot2=[]

for threshold in tqdm(range(40,100)):
    for j in range(data.shape[0]):
        predis2=np.where(predis1 > threshold/100, 1, 0)
        acc = accuracy_score(teachers,predis2)
        acc_us1 = accuracy_score(teachers[true_indexes],predis2[true_indexes])
        acc_us2 = accuracy_score(teachers[false_indexes],predis2[false_indexes])
        accplot.append((acc_us1+acc_us2)/2)
        accplot2.append(acc)
        if((acc_us1+acc_us1)/2>max_tacc):
            max_threshold=threshold/100
            max_tacc=(acc_us1+acc_us2)/2
print(max_tacc,max_threshold)
plt.plot(accplot)
plt.show()
plt.plot(accplot2)
plt.show()
plt.hist(accplot)
plt.show()

In [0]:
df_vali = pd.read_csv('test.csv')
keywords_vali = np.array(df_vali.iloc[:,[1]].fillna('nan'))
keywords_vali=keywords_vali.reshape(len(keywords_vali))
text_vali = np.array(df_vali.iloc[:,[3]])
text_vali=text_vali.reshape(len(text_vali))

x=np.stack([text_vali, keywords_vali], 1)

#testデータをid化
maxlen=158
id_=0
x_vali=np.array([[0]*maxlen]*len(x))
x_vali_key=np.array([0]*len(x))
for t in text_vali:
    tmp=text_voc.encode(t[0])
    tmp2=keyword_voc.encode([t[1]])
    for i in range(maxlen-len(tmp)):
        tmp.append(0)
    x_vali[id_]=tmp
    x_vali_key[id_]=tmp2[0]
    id_=id_+1

x_vali_=np.concatenate([x_vali.reshape(len(x_vali),maxlen), x_vali_key.reshape(len(x_vali),1)],1)
data=torch.LongTensor(x_vali_.astype(np.float32)).to('cuda')
predis1=np.array([])
model.eval()
for j in range(data.shape[0]):
    y_pred=model(data[j:j+1,:-1],data[j:j+1,-1])
    predis1=np.append(predis1,y_pred.detach().cpu().numpy())
print("threshold is:",max_threshold)
predis2=np.where(predis1 > max_threshold, 1, 0)
plt.hist(predis2)
plt.plot()

In [0]:
csv = pd.read_csv('sample_submission.csv')
csv['target']=np.array(predis2).reshape(len(predis2))
csv.to_csv('submission.csv',index=False)